<img width="10%" alt="Naas" src="https://landen.imgix.net/jtci2pxwjczr/assets/5ice39g4.png?w=160"/>

# Google Analytics - Get unique visitors by countries
<a href="https://app.naas.ai/user-redirect/naas/downloader?url=https://raw.githubusercontent.com/jupyter-naas/awesome-notebooks/master/Google%20Analytics/GoogleAnalytics_Get_stats_per_country.ipynb" target="_parent"><img src="https://naasai-public.s3.eu-west-3.amazonaws.com/open_in_naas.svg"/></a>

**Tags:** #googleanalytics #statspercountry #analytics

**Author:** [Charles Demontigny](https://www.linkedin.com/in/charles-demontigny/)

Pre-requisite: Create your own <a href="">Google API JSON credential</a>

## Input

### Import library

In [10]:
try:
    import pycountry
except:
    !pip install pycountry
    import pycountry
import plotly.graph_objects as go
import plotly.express as px
import naas
from naas_drivers import googleanalytics

### Get your credential from Google Cloud Platform

In [11]:
json_path = 'naas-googleanalytics.json'

### Get view id from google analytics

In [12]:
view_id = "228952707"

## Model

### Get trend for unique visitor by Hourly / Day / Week / Month

In [ ]:
def get_ranking(start_date, end_date, dimensions, metrics):
    df = googleanalytics.connect(json_path=json_path).views.get_data(
        view_id,
        metrics="ga:sessions",
        pivots_dimensions="ga:country",
        dimensions="ga:month",
        start_date=None,
        end_date=None,
        format_type="pivot"
    )
    return df

df = get_ranking(start_date, end_date, dimensions, metrics)

## Output

### Display result